In [ ]:
import pandas as pd

quick = True             # work with sampled data to reduce computing time
run_gridSearchCV = False # run or not hyperparameters optimization with GridSearchCV()
run_optuna = True        # run or not hyperparameters optimization with Optuna

filename = 'df-light.pkl' if quick else 'df-full.pkl'

df = pd.read_pickle(f'./{filename}')
data = df.iloc[:, 1:]
target = df['grav']

# for col in data.columns:
#     data[col] = data[col].astype('object')
# print(data.info())

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=222)

In [ ]:
from catboost import CatBoostClassifier
import shap
shap.initjs()
# Best parameters from Optuna {'classifier': 'CatBoostClassifier', 'iterations': 300, 'learning_rate': 0.09918266470302897}

model = CatBoostClassifier(iterations=300, learning_rate=0.0991, random_seed=123, cat_features=list(X_train.columns))
model.fit(X_train, y_train, verbose=True, plot=True)

In [ ]:
from shapash import SmartExplainer

xpl = SmartExplainer(
    model=model,
    # features_dict=house_dict,  # Optional parameter
    # preprocessing=encoder, # Optional: compile step can use inverse_transform method
    # postprocessing=postprocess # Optional: see tutorial postprocessing
)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
compile_df = X_test
compile_df['y_pred'] = y_pred

xpl.compile(
    x=compile_df.drop(columns=['y_pred'], axis=1),
    y_pred=compile_df['y_pred'], # Optional: for your own prediction (by default: model.predict)
    y_target=pd.Series(y_test), # Optional: allows to display True Values vs Predicted Values
)

In [ ]:
app = xpl.run_app()